In [1]:
import argparse
import glob
from hydra import compose, initialize
import hydra
import numpy as np
import pandas as pd
import torch
import yaml
from omegaconf import DictConfig, OmegaConf
from dl4cv.utils.utils import set_seed
from dl4cv.lightning_classes.plmodel import LitODModel
from dl4cv.datasets import build_taco 
from dl4cv.datasets.taco_data import taco_val_test_collate_fn
from tqdm import tqdm
import os


: 

In [2]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path="./configs")
inference_cfg = compose(config_name="config_taco_training")
inference_cfg["inference"]["run_name"] = 'Resnet50_Whale'
inference_cfg["inference"]["device"] = 'cuda:0'
path = f"outputs/{inference_cfg.inference.run_name}/.hydra/config.yaml"
with open(path) as cfg:
    cfg_yaml = yaml.safe_load(cfg)
cfg_yaml["inference"] = inference_cfg["inference"]
cfg_yaml["datamodule"]["test"]["params"]["num_to_return"] = 100
cfg = OmegaConf.create(cfg_yaml)


In [3]:
def main(cfg: DictConfig) -> None:
    """
    Run pytorch-lightning model inference
    Args:
        cfg: hydra config
    Returns:
        None
    """
    set_seed(cfg.training.seed)

    device = torch.device(cfg.inference.device)

    model_names = glob.glob(f"outputs/{cfg.inference.run_name}/saved_models/*.ckpt")
    _, _, test_set = build_taco(cfg)
    # Dirty trick to get the ground truth boxes
    loader = torch.utils.data.DataLoader(
        test_set,
        collate_fn=taco_val_test_collate_fn,
        batch_size=1,
        num_workers=1,
        shuffle=False,
    )
    lit_model = LitODModel.load_from_checkpoint(checkpoint_path=model_names[0], cfg=cfg)
    lit_model.to(device)
    predictions = []
    for batch in tqdm(loader):
        # move batch elements to device
        batch = tuple(b.to(device) for b in batch)
        predictions.append(
            lit_model.nms_on_image(
                batch,
            )
        )

    return predictions




In [4]:
predictions = main(cfg)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [02:30<00:00,  1.66it/s]


In [5]:
print(len(predictions[6]))

9


In [6]:
import json
import numpy as np
import os
import cv2
from tqdm import tqdm
from PIL import Image, ExifTags
from dl4cv.utils.object_detect_utils import get_iou, fix_orientation, CAT_MAPPING

def _get_ground_truth_boxes():
    file = '/dtu/blackhole/0f/160495/s210527/taco/test/test_annotations.json'
    with open(file, "r") as f:
        dataset = json.load(f)
    
    target_size = (1280, 720)
    BACKGROUND_LABEL = -1
    label_map = CAT_MAPPING

    for orientation in ExifTags.TAGS.keys():
        if ExifTags.TAGS[orientation] == "Orientation":
            break
    dataset_root_path="/dtu/datasets1/02514/data_wastedetection"
    
    out_gt = {}
    for i in dataset["images"]:
        print(i)
        annot_boxes = [
            annotation["bbox"]
            for annotation in dataset["annotations"]
            if annotation["image_id"] == i['image_id']
        ]
        annots_labels = [
            label_map[annotation["category_id"]]
            for annotation in dataset["annotations"]
            if annotation["image_id"] == i['image_id']
        ]
        filename = os.path.join(dataset_root_path, i["file_name"])
        
        I = fix_orientation(filename, orientation)
        image = cv2.cvtColor(np.array(I), cv2.COLOR_RGB2BGR)
        scale_factor = (
            image.shape[0] / target_size[1],  # height
            image.shape[1] / target_size[0],  # width
        )
        image = cv2.resize(
            image, (target_size[0], target_size[1]), interpolation=cv2.INTER_AREA
        )
        gtvalues = []
        for a_index, annot in enumerate(annot_boxes):
            x1 = int(annot[0] / scale_factor[1])
            y1 = int(annot[1] / scale_factor[0])
            x2 = int(annot[0] / scale_factor[1] + annot[2] / scale_factor[1])
            y2 = int(annot[1] / scale_factor[0] + annot[3] / scale_factor[0])
            assert (
                y2 <= target_size[1] and y1 <= target_size[1]
            ), f"y2 = {y2} and y1 = {y1} for gt of image {filename} with resized dims {image.shape}"
            assert (
                x2 <= target_size[0] and x1 <= target_size[0]
            ), f"x2 = {x2} and x1 = {x1} for gt of image {filename} with resized dims {image.shape}"
            gtvalues.append(
                {
                    "x1": x1,
                    "x2": x2,
                    "y1": y1,
                    "y2": y2,
                    "label": annots_labels[a_index],
                }
            )
        out_gt[i['image_id']] = gtvalues

    return out_gt


In [7]:
gt_boxes = _get_ground_truth_boxes()

KeyError: 'image_id'

In [ ]:
predictions[6]

[{'bbox': {'x1': tensor(723, device='cuda:0', dtype=torch.int32),
   'y1': tensor(158, device='cuda:0', dtype=torch.int32),
   'x2': tensor(958, device='cuda:0', dtype=torch.int32),
   'y2': tensor(695, device='cuda:0', dtype=torch.int32)},
  'conf': 0.9853888154029846,
  'pred_class': 20,
  'true_class': 20,
  'image_id': 217},
 {'bbox': {'x1': tensor(188, device='cuda:0', dtype=torch.int32),
   'y1': tensor(99, device='cuda:0', dtype=torch.int32),
   'x2': tensor(548, device='cuda:0', dtype=torch.int32),
   'y2': tensor(395, device='cuda:0', dtype=torch.int32)},
  'conf': 0.9745054244995117,
  'pred_class': 13,
  'true_class': 13,
  'image_id': 217},
 {'bbox': {'x1': tensor(224, device='cuda:0', dtype=torch.int32),
   'y1': tensor(261, device='cuda:0', dtype=torch.int32),
   'x2': tensor(236, device='cuda:0', dtype=torch.int32),
   'y2': tensor(271, device='cuda:0', dtype=torch.int32)},
  'conf': 0.827072262763977,
  'pred_class': 14,
  'true_class': 28,
  'image_id': 217},
 {'bbox':

: 

: 

: 